In [13]:
!pip install ydata-profiling


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd
import numpy as np
import pyodbc
import pandas as pd
from ydata_profiling import ProfileReport
import os
os.environ["OMP_NUM_THREADS"] = "1"

engine = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-BCE45FF\\MSSQLSERVERE;"
    "DATABASE=Airline_DW;"
    "Trusted_Connection=yes;"
)



dim_partner = pd.read_sql("SELECT * FROM DimPartner", engine)
fact_partner = pd.read_sql("SELECT * FROM FactPartner", engine)

C:\Users\emna1\AppData\Local\Temp\ipykernel_41420\263950804.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_partner = pd.read_sql("SELECT * FROM DimPartner", engine)
C:\Users\emna1\AppData\Local\Temp\ipykernel_41420\263950804.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fact_partner = pd.read_sql("SELECT * FROM FactPartner", engine)


In [15]:
print(dim_partner.columns)
print(fact_partner.columns)

Index(['Partner_key', 'PartnerName', 'PartnerType', 'CostPerPointIssued'], dtype='object')
Index(['Partner_key', 'Date_key', 'IssuedPoints', 'TotalCostOfIssuedPoints'], dtype='object')


In [16]:
df_partner = fact_partner.merge(
    dim_partner,
    on='Partner_key',
    how='left'
)
df_partner.head(19660)

,Partner_key,Date_key,IssuedPoints,TotalCostOfIssuedPoints,PartnerName,PartnerType,CostPerPointIssued
0,50,20231004,482458,8394.77,Glacier Hostel,Hotel,0.01
1,51,20231008,450051,6165.70,Glacier Hotel Group 34,Hotel,0.01
2,52,20231019,452974,7836.45,Glacier Ride,CarRental,0.01
3,53,20231003,262666,3624.79,Glacier Suites 26,Hotel,0.01
4,54,20231006,352270,5988.59,Glacier TrustCard 30,Financial,0.02
...,...,...,...,...,...,...,...
19655,56,20180529,441672,5962.57,Granite Boutique,Hotel,0.01
19656,57,20180518,685791,9669.65,Granite Electronics,eStore,0.01
19657,58,20180517,343331,5218.63,Granite Home,eStore,0.01
19658,59,20180529,594331,8320.63,Granite Hostel,Hotel,0.01


In [17]:
df_agg = df_partner.groupby('Partner_key').agg(
    PartnerName=('PartnerName', 'first'),          # Garde le nom
    PartnerType=('PartnerType', 'first'),          # Garde le type
    TotalIssuedPoints=('IssuedPoints', 'sum'),    # Somme des points émis
    TotalCost=('TotalCostOfIssuedPoints', 'sum'), # Somme des coûts
    AvgCostPerPoint=('CostPerPointIssued', 'mean')# Moyenne du coût par point
).reset_index()




In [18]:
df_agg.head(200)



,Partner_key,PartnerName,PartnerType,TotalIssuedPoints,TotalCost,AvgCostPerPoint
0,1,Aurora Auto,CarRental,386613050,7191002.74,0.01
1,2,Aurora Aviation 22,Airline,412854148,10114926.62,0.01
2,3,Aurora Bank,Financial,402120444,7921772.62,0.01
3,4,Aurora Boutique,Hotel,411429236,9956587.30,0.01
4,5,Aurora Car Hire 21,CarRental,416145566,8281296.80,0.01
...,...,...,...,...,...,...
195,196,TrueNorth Outdoors 33,eStore,53339834,848103.32,0.01
196,197,TrueNorth Outfitters,eStore,46477798,594915.90,0.01
197,198,TrueNorth Ride 25,CarRental,48508132,698517.02,0.01
198,199,TrueNorth Sports 35,eStore,50038866,710551.94,0.01


In [19]:
df_agg = df_agg.drop(
    columns=['Partner_key', 'PartnerName', 'PartnerType', 'AvgCostPerPoint']
)



In [20]:
profile = ProfileReport(df_agg, title=" Dataset Report")
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Verify
np.isnan(X_scaled).sum(), np.isinf(X_scaled).sum()


(0, 0)

In [22]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df_agg["Cluster"] = kmeans.fit_predict(X_scaled)


In [23]:
df_agg["CostPerPoint"] = (
    df_agg["TotalCost"] / df_agg["TotalIssuedPoints"]
)

summary = df_agg.groupby("Cluster").mean()
summary


,TotalIssuedPoints,TotalCost,CostPerPoint
Cluster,,,
0,5.436348e+07,7.856989e+05,0.014269
1,4.013425e+08,8.754757e+06,0.021805


In [24]:
best_cluster = summary["CostPerPoint"].idxmin()

df_agg["Performance"] = df_agg["Cluster"].apply(
    lambda c: "Performing" if c == best_cluster else "Underperforming"
)


In [25]:
performance_summary = df_agg["Performance"].value_counts().reset_index()
performance_summary.columns = ["Performance", "Count"]
performance_summary["Percentage"] = (performance_summary["Count"] / performance_summary["Count"].sum() * 100).round(2)

print(performance_summary)



       Performance  Count  Percentage
0       Performing    160        80.0
1  Underperforming     40        20.0


In [26]:
import joblib

joblib.dump(kmeans, "ClusteringPartners.pkl")

print("✅ Clustering model saved")


✅ Clustering model saved
